In [7]:
import pandas as pd
import prophet as ph
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

# DataFrame 생성
all_forecasts = pd.DataFrame()

# 엑셀파일에서 데이터 가져오기.
file_path = 'D:\Git\Predict_workload\Dataset.xlsx'
columns_to_select = ['ITEM_CODE', 'PRODUCT_WPNL', 'CLOCK_ACCEPT', 'RECIPE']

df = pd.read_excel(file_path, usecols=columns_to_select)

item_codes = df['ITEM_CODE'].unique()

model = ph.Prophet()
        
for item in item_codes:
    # 학습된 모델 불러오기
    with open('D:\Git\Predict_workload\prophet_model.pkl', 'rb') as f:
        model = pickle.load(f)
        
        # 다음날 하루 (1일) 생산량 예측
        future = model.make_future_dataframe(periods=30, freq='D')
        
        # 예측
        forecast = model.predict(future)
        
        # 예측 데이터에 ITEM 정보 추가
        forecast['ITEM_CODE'] = item
        all_forecasts = pd.concat([all_forecasts, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'ITEM_CODE']]], ignore_index=True)
        
        # 실제 데이터와 예측데이터 병합
        merged = pd.merge(df, forecast[['ds', 'yhat']], on='ds', how='left')
        
         # 오차 계산
        mae = mean_absolute_error(merged['y'], merged['yhat'])
        mse = mean_squared_error(merged['y'], merged['yhat'])
        results = []
        results.append({'ITEM_CODE': item, 'MAE': mae, 'MSE': mse})
        
# 결과 출력
results_df = pd.DataFrame(results)
print(results_df)

# yhat = trend + daily + other components(weekly, lower, Hight 등등)
output_file_path = 'D:\Git\Predict_workload\daily_predicted_production_times.xlsx'
all_forecasts.to_excel(output_file_path, index=False)
print(f'Predictions saved to {output_file_path}')

# 예측 결과 엑셀로 내보내기
output_file_path_1 = 'D:\Git\Predict_workload\predicted_production_times_evaluation.xlsx'
results_df.to_excel(output_file_path_1, index=False)



KeyError: 'ds'